In [1]:
from typing import Tuple

import itertools
import logging

import numpy
import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

In [2]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('prophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('fbprophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [3]:
country = 'italy'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [4]:
def make_dataset(df_processed: pandas.DataFrame) -> pandas.DataFrame:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)

    # unemployment rate, being a percentage, can only be between 0 to 100
    df_mrd['floor'] = 0
    df_mrd['cap'] = 100
    return df_mrd

In [5]:
df_mrd = make_dataset(df)

In [6]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [7]:
df_train, df_test = train_test_split(df_mrd, 12)

In [8]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse'):
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # other params
    model_params = {
        'growth': 'logistic', 'seasonality_mode': 'additive',
        'daily_seasonality': False, 'weekly_seasonality': False, 'yearly_seasonality': False
    }
    cv_params = {'horizon': '30 days', 'parallel': 'processes'}
    pm_params = {'metrics': [eval_metric, 'coverage'], 'rolling_window': 0.1, 'monthly': True}

    print(f'Total hyper-parameter set count: {len(all_params)}')

    # Use cross validation to evaluate all parameters
    iteration_count: int = 1
    for hyper_params in all_params:
        print(f'Set: {iteration_count}')
        print(f'{hyper_params}')

        m = Prophet(**hyper_params, **model_params).add_seasonality(
            name='quarterly', period=365.25/4, fourier_order = 5
            ).fit(df)
        df_cv = cross_validation(m, **cv_params)
        df_p = performance_metrics(df_cv, **pm_params)
        eval_metric_values.append(df_p[eval_metric].values[0])

        print(f'{eval_metric.upper()}: {df_p[eval_metric].values[0]}')
        iteration_count += 1
        print()

    best_params: dict = all_params[numpy.argmin(eval_metric_values)]

    print('Training model on the best hyper-parameter set.')
    print(f'{best_params}')
    
    auto_model: Prophet = Prophet(**best_params, **model_params).add_seasonality(
        name='quarterly', period=365.25/4, fourier_order = 5
        ).fit(df)

    print('Cross-Validating best model.')
    auto_model_cv = cross_validation(auto_model, **cv_params)
    auto_model_p: pandas.DataFrame = performance_metrics(auto_model_cv, **pm_params)

    return auto_model, auto_model_p, best_params

In [9]:
param_grid = {  
    'changepoint_prior_scale': [0.5, 0.75],
    'seasonality_prior_scale': [0.1, 1.0],
}

auto_model, auto_model_p, best_params = train_model(df_train, param_grid)

Total hyper-parameter set count: 4
Set: 1
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 0.8229110245091099

Set: 2
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 1.0}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 0.8257373144357424

Set: 3
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 0.1}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 0.8763551801598506

Set: 4
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 1.0}
RMSE: 0.8147150999897648

Training model on the best hyper-parameter set.
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 1.0}
Cross-Validating best model.


In [10]:
best_params

{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 1.0}

In [11]:
auto_model_p

,horizon,rmse,coverage
0,1,0.814715,0.200883
1,2,0.789446,0.209616


In [12]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [13]:
df_predicted, rmse = test_model(df_test, auto_model)

In [14]:
df_predicted

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,quarterly,quarterly_lower,quarterly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,16.065855,100,0,8.470591,9.194736,16.065855,16.065855,-7.250823,-7.250823,-7.250823,-7.250823,-7.250823,-7.250823,0.0,0.0,0.0,8.815032
1,2022-04-01,16.025901,100,0,8.350861,9.060665,16.026009,16.026009,-7.317204,-7.317204,-7.317204,-7.317204,-7.317204,-7.317204,0.0,0.0,0.0,8.708697
2,2022-05-01,15.987313,100,0,8.361441,9.092774,15.987632,15.987632,-7.263891,-7.263891,-7.263891,-7.263891,-7.263891,-7.263891,0.0,0.0,0.0,8.723422
3,2022-06-01,15.947518,100,0,8.190598,8.947807,15.947929,15.947929,-7.370223,-7.370223,-7.370223,-7.370223,-7.370223,-7.370223,0.0,0.0,0.0,8.577295
4,2022-07-01,15.909082,100,0,8.242190,8.972435,15.909781,15.909781,-7.297062,-7.297062,-7.297062,-7.297062,-7.297062,-7.297062,0.0,0.0,0.0,8.612020
5,2022-08-01,15.869445,100,0,8.140749,8.889291,15.855914,15.886204,-7.342743,-7.342743,-7.342743,-7.342743,-7.342743,-7.342743,0.0,0.0,0.0,8.526702
6,2022-09-01,15.829888,100,0,8.165769,8.932219,15.795406,15.860300,-7.269314,-7.269314,-7.269314,-7.269314,-7.269314,-7.269314,0.0,0.0,0.0,8.560573
7,2022-10-01,15.791683,100,0,8.099229,8.887697,15.730355,15.839281,-7.313633,-7.313633,-7.313633,-7.313633,-7.313633,-7.313633,0.0,0.0,0.0,8.478049
8,2022-11-01,15.752283,100,0,8.030231,8.819757,15.664372,15.836942,-7.311267,-7.311267,-7.311267,-7.311267,-7.311267,-7.311267,0.0,0.0,0.0,8.441016
9,2022-12-01,15.714230,100,0,7.978397,8.800250,15.589774,15.835807,-7.345077,-7.345077,-7.345077,-7.345077,-7.345077,-7.345077,0.0,0.0,0.0,8.369153


In [15]:
rmse

0.527592481166063

In [16]:
with open(f'{country}_prophet_cv_model_2.json', 'w') as f:
    f.write(model_to_json(auto_model))

In [17]:
df_future = pandas.DataFrame(data = {
        'ds': ['2023-03-01', '2023-04-01', '2023-05-01'],
        'floor': [0, 0, 0],
        'cap': [100, 100, 100],
    }
)

In [18]:
df_future_prediction = auto_model.predict(df_future)

In [19]:
df_future_prediction

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,quarterly,quarterly_lower,quarterly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,15.600521,100,0,8.086292,8.773015,15.600521,15.600521,-7.161547,-7.161547,-7.161547,-7.161547,-7.161547,-7.161547,0.0,0.0,0.0,8.438974
1,2023-04-01,15.561510,100,0,7.890333,8.624925,15.561778,15.561778,-7.302733,-7.302733,-7.302733,-7.302733,-7.302733,-7.302733,0.0,0.0,0.0,8.258777
2,2023-05-01,15.523833,100,0,7.978348,8.694199,15.524856,15.524856,-7.198338,-7.198338,-7.198338,-7.198338,-7.198338,-7.198338,0.0,0.0,0.0,8.325495
